# Data search

<div class="alert alert-block alert-info">
<b> purpose </b> = get ids from the activities that we will need in the foreground database
    <div>

for the foreground, we need the following activites :
- wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future
- operation, solar collector system, Cu flat plate collector, one-family house, for hot water
- electricity production, photovoltaic, 3kWp slanted-roof 
- electricity production, wind, 1-3MW turbine, onshore
- electricity production, hydro, run-of-river
- heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
import numpy as np

if 'ei38-teaching-25' not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")
    
bd.projects.set_current('ei38-teaching-25')

In [2]:
db = bd.Database("ei 3.8 cutoff")

## heating activites

Basically we reach in the ecoinvent database the activity we want

In [5]:
database_name = "ei 3.8 cutoff"
wood_pellet = bd.get_activity(
    database=database_name, 
    name='wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future', 
    unit = "megajoule"
    )

Thus, we have the id by doing this

In [6]:
wood_pellet.id

19364

In [9]:
db.search('operation, solar collector system, Cu flat plate collector, one-family house, for hot water')

['operation, solar collector system, Cu flat plate collector, one-family house, for hot water' (megajoule, RoW, None),
 'operation, solar collector system, Cu flat plate collector, one-family house, for hot water' (megajoule, CH, None),
 'operation, solar collector system, Cu flat plate collector, one-family house, for hot water' (megajoule, CA-QC, None)]

In [10]:
solar_panels = bd.get_activity(
    database=database_name, 
    name='operation, solar collector system, Cu flat plate collector, one-family house, for hot water', 
    location='CH',
    )

In [11]:
solar_panels.id

8302

## electricity activities

I can get the code directly from Activity Browser which is more practicale

In [52]:
PV=db.get('eefb3fb50de3c1b579b04b113644f681')
PV.id

18352

In [53]:
Wind=db.get('db261be24b09c401a5a88b5af3243f65')
Wind.id

10319

In [54]:
Hydro=db.get('ef7d96b8b3b9eabc22bc37d7368e5385')
Hydro.id

6599

In [55]:
Biomass=db.get('bc1a68a31dbcaf326cefb48b71b2982a')
Biomass.id

20079

## Export id

In [59]:
wood_pellet['name']

'wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future'

In [62]:
dict_export={
    wood_pellet['name']:wood_pellet.id,
    solar_panels['name']:solar_panels.id,
    PV['name']:PV.id,
    Wind['name']:Wind.id,
    Hydro['name']:Hydro.id,
    Biomass['name']:Biomass.id
}

In [63]:
import json

with open("./data/inputs/dict_id.json", "w") as outfile:
    json.dump(dict_export, outfile)

## identify the activity name from ids of the foreground

In [68]:
# if "foreground" in bd.databases:
#     del bd.databases["foreground"]
    
foreground_importer = bi.ExcelImporter("./data/inputs/lci_moeschberg.xlsx")
# for act in foreground_importer.data:
#     # print(act["name"])
#     for exc in act["exchanges"]:
#         if "classifications" in exc:
#             exc.pop("classifications")
#         if "properties" in exc:
#             exc.pop("properties")   
foreground_importer.apply_strategies()
foreground_importer.match_database("biosphere3", fields=("name", "unit", "categories"))
foreground_importer.match_database("ei 3.8 cutoff", fields=("name", "unit", "location"))
foreground_importer.statistics()
foreground_importer.write_database()

Extracted 1 worksheets in 0.00 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 7.03 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
5 datasets
18 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 da

In [3]:
fu = {bd.get_activity(database="energy_moeschberg", name = "energy demand, operational, Hotel Moeschberg"):1}

In [4]:
### it returns functional_unit, data_objects, and a mapping dictionary
fu, objects, remapping_dicts = bd.prepare_lca_inputs(fu, method=('IPCC 2013', 'climate change', 'GWP 100a'))


In [5]:
data, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.data")
indices, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.indices")
flip, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.flip")
unique_indices = set([a for b in indices for a in b])
mapping_act=dict.fromkeys(int(i) for i in unique_indices) #json accepts only int not int32
for i in unique_indices:
	mapping_act[i]=bd.get_activity(i)['name']
rows = [tup[0] for tup in indices]
cols = [tup[1] for tup in indices]


In [6]:
import pandas as pd
matrix_raw = pd.DataFrame({"row":rows,
                       	"col":cols,
                       	"from":[mapping_act[idx] for idx in rows],
                       	"to":[mapping_act[idx] for idx in cols],
                       	"data":data
                      	})
matrix = matrix_raw.pivot(index='row',columns='col', values='data').fillna(0)
# mapping
matrix_raw


,row,col,from,to,data
0,6599,23994,"electricity production, hydro, run-of-river","electricity supply, Energie Grosshoechstetten ...",0.907000
1,6599,23995,"electricity production, hydro, run-of-river","electricity, subsidised, 2021",0.320000
2,8302,23996,"operation, solar collector system, Cu flat pla...","heat supply, Hotel Moeschberg, 2021",0.004668
3,10319,23995,"electricity production, wind, 1-3MW turbine, o...","electricity, subsidised, 2021",0.020000
4,18352,23993,"electricity production, photovoltaic, 3kWp sla...","electricity supply, Hotel Moeschberg, 2021",0.440600
5,18352,23994,"electricity production, photovoltaic, 3kWp sla...","electricity supply, Energie Grosshoechstetten ...",0.026000
6,18352,23995,"electricity production, photovoltaic, 3kWp sla...","electricity, subsidised, 2021",0.460000
7,20079,23995,"heat and power co-generation, wood chips, 6667...","electricity, subsidised, 2021",0.200000
8,23758,23996,"heat production, wood pellet, at furnace 25kW,...","heat supply, Hotel Moeschberg, 2021",0.995300
9,23993,23993,"electricity supply, Hotel Moeschberg, 2021","electricity supply, Hotel Moeschberg, 2021",1.000000


In [58]:
new_arrays = {
	(18352,23993): [0.4406,1], #PV becomes dominant
	(23994,23993): [0.5594,0],
	(23758,23996): [0.995300,0], #wood pellets are replaced by solar thermal
	(8302,23996): [0.004668,1],
        	}

In [7]:
indices

array([( 6599, 23994), ( 6599, 23995), ( 8302, 23996), (10319, 23995),
       (18352, 23993), (18352, 23994), (18352, 23995), (20079, 23995),
       (23758, 23996), (23993, 23993), (23993, 23997), (23994, 23993),
       (23994, 23994), (23995, 23994), (23995, 23995), (23996, 23996),
       (23996, 23997), (23997, 23997)],
      dtype=[('row', '<i4'), ('col', '<i4')])

In [8]:
unique_indices = set([a for b in indices for a in b])

In [9]:
# dict_act=dict.fromkeys(int(i) for i in unique_indices) #json accepts only int not int32
# for i in unique_indices:
#     dict_act[i]=bd.get_activity(i)['name']

In [15]:
bd.get_activity(6599)['reference product']

'electricity, high voltage'

In [31]:
a=pd.DataFrame({'hello':[1,2,3]})
a

,hello
0,1
1,2
2,3


In [33]:
a['hello'][1]

2

In [34]:
df = pd.DataFrame(columns=['id','name','location','reference product'],index=np.arange(len(unique_indices)))
n=0
for i in unique_indices: 
    df['id'][n]=(i)
    df['name'][n]=(bd.get_activity(i)['name'])
    df['location'][n]=(bd.get_activity(i)['location'])
    df['reference product'][n]=(bd.get_activity(i)['reference product'])
    n += 1

In [35]:
df

,id,name,location,reference product
0,6599,"electricity production, hydro, run-of-river",CH,"electricity, high voltage"
1,8302,"operation, solar collector system, Cu flat pla...",CH,"heat, central or small-scale, other than natur..."
2,10319,"electricity production, wind, 1-3MW turbine, o...",CH,"electricity, high voltage"
3,18352,"electricity production, photovoltaic, 3kWp sla...",CH,"electricity, low voltage"
4,20079,"heat and power co-generation, wood chips, 6667...",CH,"electricity, high voltage"
5,23758,"heat production, wood pellet, at furnace 25kW,...",CH,"heat, central or small-scale, other than natur..."
6,23993,"electricity supply, Hotel Moeschberg, 2021",CH,"electricity, low voltage"
7,23994,"electricity supply, Energie Grosshoechstetten ...",CH,"electricity, low voltage"
8,23995,"electricity, subsidised, 2021",CH,"electricity, low voltage"
9,23996,"heat supply, Hotel Moeschberg, 2021",CH,"heat, Hotel Moeschberg"


In [38]:
bd.get_activity(product='lubricating oil')

MultipleResults: Found 4 results for the given search